In [1]:
import requests
import json
import urllib.parse
import sys
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
class Disease:
    """
    Description: A class to represent a disease.
    """
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name

    def find_genes(self, species):
        """
        Description: Find the genes associated with the disease.
        Query MyGene.info to find the associated gene symbols.
        :return: A list of gene symbols
        """
        # Define the API endpoint
        mygene_url = 'http://mygene.info/v3/query'
        # Define the API parameters
        mygene_params = {
            'q': self.name,
            'fields': 'symbol,name,summary',
            'species': species,
            'size': 10  # maximum number of results to return
            }
        # Send the GET request to the API and retrieve the response
        mygene_response = requests.get(mygene_url, params=mygene_params)
        # Check if the request was successful
        if not mygene_response.ok:
            # Raise an exception if the request was unsuccessful
            mygene_response.raise_for_status()
        # Retrieve the JSON data from the response
        mygene_data = mygene_response.json()
        genes = []
        # Extract gene information from the response
        for res in mygene_data['hits']:
            gene_symbol = res['symbol']
            genes.append(Gene(species, gene_symbol))
        return genes

In [3]:
class Gene:
    """
    Description: A class to represent a gene.
    """

    def __init__(self, species, symbol, id=None):
        self.species = species
        self.symbol = symbol
        if id is None:
            self.id = self.get_id()
        else:
            self.id = id

    def __str__(self):
        return self.species + ' - ' + self.symbol + ' - ' + self.id

    def get_id(self):
        """
        ATTENTION: ONLY WORKS FOR HUMAN GENES!
        Description: Gets the Ensembl gene ID from the gene symbol.
        Query Ensembl to convert the gene symbol to an Ensembl gene ID.
        :return: An Ensembl gene ID
        """
        ensembl_url = 'https://rest.ensembl.org/xrefs/symbol/homo_sapiens/'
        # Define the API parameters
        ensembl_params = {
            'content-type': 'application/json',
            'object_type': 'gene',
            'ensembl_release': 'GRCh38'
        }
        # Send the GET request to the API and retrieve the response
        ensembl_response = requests.get(ensembl_url + self.symbol, params=ensembl_params)
        # Check if the request was successful
        if not ensembl_response.ok:
            # Raise an exception if the request was unsuccessful
            ensembl_response.raise_for_status()
        # Retrieve the JSON data from the response
        ensembl_data = ensembl_response.json()
        # Raise an exception if no Ensembl gene ID was found or multiple Ensembl gene IDs were found
        if len(ensembl_data) == 0:
            # Raise an exception that no Ensembl gene ID was found
            raise Exception('No Ensembl gene ID was found for the gene symbol: ' + self.symbol)
        # There might be multiple Ensembl gene IDs for the gene symbol, but we are ignoring it
        # We are returning only the first Ensembl gene ID
        return ensembl_data[0]['id']

    def get_orthologues(self, target_species):
        """
        Description: Gets the orthologues of the gene for the species of interest.
        :param species1: The species of the starting gene
        :param species2: The species of the orthologues
        :return:
        """
        homology_url = 'https://rest.ensembl.org/homology/id/'
        # Define the API parameters
        params = {
            'content-type': 'application/json',
            'type': 'orthologues',
            'target_species': target_species
        }
        # Send the GET request to the API and retrieve the response
        homology_response = requests.get(homology_url + self.id, params=params)
        # Check if the request was successful
        if not homology_response.ok:
            # Raise an exception if the request was unsuccessful
            homology_response.raise_for_status()
        # Retrieve the JSON data from the response
        homology_data = homology_response.json()
        # Extract the orthologues gene IDs from the response
        for homology in homology_data["data"][0]["homologies"]:
            if homology['type'] == 'ortholog_one2one':
                return Gene(target_species, self.symbol, homology["target"]["id"])
        return None

    def get_proteins(self):
        """
        Description: Gets the proteins of the gene.
        Query UniProt to get the proteins of the gene.
        :return:
        """
        # Use UniProt to find the proteins produced by the orthologues
        uniprot_url = f"https://rest.uniprot.org/uniprotkb/stream?compressed=false&format=json&query=%28{self.species}%20{self.id}%29"
        uniprot_response = requests.get(uniprot_url)
        # Check if the request was successful\
        if not uniprot_response.ok:
            # Raise an exception if the request was unsuccessful
            uniprot_response.raise_for_status()
        # convert response.content from bytes to dictionary
        data = json.loads(uniprot_response.content)
        # If the data is empty, skip it
        if len(data["results"]) == 0:
            return None
        return Protein(self.species, self.symbol, data["results"][0]["primaryAccession"], data["results"][0]["sequence"]["value"])

In [4]:
class Protein:
    """
    Description: A class to represent a protein.
    """
    def __init__(self, species, name, uniprot_id, sequence):
        self.species = species
        self.name = name
        self.uniprot_id = uniprot_id
        self.sequence = sequence

    def __str__(self):
        return self.species + ' - ' + self.name + ' - ' + self.uniprot_id + ' - ' + self.sequence

In [21]:
# Example 1: Get the genes associated with the disease
bd = Disease('Rheumatoid arthritis')
print(bd)
bd_h_genes = bd.find_genes('human')
for bd_h_gene in bd_h_genes:
    print(bd_h_gene)
    print(bd_h_gene.get_orthologues('mus_musculus'))

Rheumatoid arthritis
human - SERPINH1 - ENSG00000149257
mus_musculus - SERPINH1 - ENSMUSG00000070436
human - BST1 - ENSG00000197121
mus_musculus - BST1 - ENSMUSG00000073678
human - MACIR - ENSG00000181751
mus_musculus - MACIR - ENSMUSG00000044768
human - LECT2 - ENSG00000145826
mus_musculus - LECT2 - ENSMUSG00000021539
human - OLAH - ENSG00000152463
mus_musculus - OLAH - ENSMUSG00000026645
human - BTLA - ENSG00000186265
mus_musculus - BTLA - ENSMUSG00000052013
human - TAGAP - ENSG00000164691
None
human - FSTL1 - ENSG00000163430
mus_musculus - FSTL1 - ENSMUSG00000022816
human - SAA2 - ENSG00000134339
None
human - ZNF334 - ENSG00000198185
mus_musculus - ZNF334 - ENSMUSG00000017667


In [7]:
# Example 1: Get the human SOX9 gene and its orthologues in mouse
h_sox9 = Gene('human', 'SOX9')
print(h_sox9)
m_sox9 = h_sox9.get_orthologues('mus_musculus')
print(m_sox9)
h_sox9_p = h_sox9.get_proteins()
print(h_sox9_p)

human - SOX9 - ENSG00000125398
mus_musculus - SOX9 - ENSMUSG00000000567
human - SOX9 - P48436 - MNLLDPFMKMTDEQEKGLSGAPSPTMSEDSAGSPCPSGSGSDTENTRPQENTFPKGEPDLKKESEEDKFPVCIREAVSQVLKGYDWTLVPMPVRVNGSSKNKPHVKRPMNAFMVWAQAARRKLADQYPHLHNAELSKTLGKLWRLLNESEKRPFVEEAERLRVQHKKDHPDYKYQPRRRKSVKNGQAEAEEATEQTHISPNAIFKALQADSPHSSSGMSEVHSPGEHSGQSQGPPTPPTTPKTDVQPGKADLKREGRPLPEGGRQPPIDFRDVDIGELSSDVISNIETFDVNEFDQYLPPNGHPGVPATHGQVTYTGSYGISSTAATPASAGHVWMSKQQAPPPPPQQPPQAPPAPQAPPQPQAAPPQQPAAPPQQPQAHTLTTLSSEPGQSQRTHIKTEQLSPSHYSEQQQHSPQQIAYSPFNLPHYSPSYPPITRSQYDYTDHQNSSSYYSHAAGQGTGLYSTFTYMNPAQRPMYTPIADTSGVPSIPQTHSPQHWEQPVYTQLTRP


In [37]:
# Save the data for Knowledge Graph

disease_list = [Disease('bipolar disorder'), Disease('high blood pressure'), Disease('cardiovascular disease'), Disease('Rheumatoid arthritis'), Disease('Anxiety disorders')]
data = {}
for disease in disease_list:
    data[disease.name] = {}
    data[disease.name]['human_gene'] = {}
    data[disease.name]['mouse_gene'] = {}
    data[disease.name]['human_protein'] = {}
    data[disease.name]['mouse_protein'] = {}
    h_genes = []
    h_proteins = []
    m_genes = []
    m_proteins = []
    for h_gene in disease.find_genes('human'):
        m_gene = h_gene.get_orthologues('mus_musculus')
        if m_gene is not None:
            h_genes.append(h_gene)
            m_genes.append(m_gene)
            m_protein = m_gene.get_proteins()
            m_proteins.append(m_protein)
            h_protein = h_gene.get_proteins()
            h_proteins.append(h_protein)
    data[disease.name]['human_gene'] = [(h_gene.symbol, h_gene.id) for h_gene in h_genes]
    data[disease.name]['mouse_gene'] = [(m_gene.symbol, m_gene.id) for m_gene in m_genes]
    data[disease.name]['human_protein'] = [(h_protein.name, h_protein.uniprot_id, h_protein.sequence) for h_protein in h_proteins]
    data[disease.name]['mouse_protein'] = [(m_protein.name, m_protein.uniprot_id, m_protein.sequence) for m_protein in m_proteins]

In [ ]:
# Create an empty graph
M = nx.Graph()

# Read mock dataset
# iterate through keys of the dictionary (disease)
for disease in data.keys():
    M.add_node(disease, type="Disease") # print disease name
    disease_data = data[disease]  # get the dictionary of the disease, which contains the gene and protein data
    for element in disease_data.keys():  # iterate through the keys of the disease dictionary, i.e. the gene and protein data
        # print(element)  # print the key of the dictionary, e.g. human_gene
        species_type, type = element.split('_')  # split the key into species and type
        # print(species, type)
        element_list = disease_data[element]  # get the list of genes or proteins
        #print(len(element_list)) 
        for i, item in enumerate(element_list):  # iterate through the list of genes or proteins
            # print(item)
            if type == 'gene':  # if the type is gene, then the item is a tuple of gene symbol and gene id
                gene_symbol, gene_id = item
                #print(gene_id, gene_symbol, species_type)
                # check if node is already present in the graph
                if gene_id not in M.nodes:
                    M.add_node(gene_id, name = gene_symbol, species = species_type, type= "Gene") 
                if species_type == "human":
                    M.add_edge(gene_id, disease, type="Association")
                elif species_type == "mouse":
                    mouse_gene = data[disease][species_type + '_gene'][i]
                    human_gene = data[disease]['human_gene'][i]
                    M.add_edge(mouse_gene[1], human_gene[1], type="Orthology")

                # check if the gene is already present in the nodes of the graph
                #if gene_symbol in G.nodes:
                    # if the gene is already present, then add the disease as a node attribute
                   # G.nodes[gene_symbol]['disease'] = disease
                # print(gene_symbol, gene_id)
            elif type == 'protein':  # if the type is protein, then the item is a tuple of protein name, protein id and protein sequence
                protein_symbol, protein_id, protein_sequence = item 
                if protein_id not in M.nodes:
                    M.add_node(protein_id, name = protein_symbol, species = species_type, sequence = protein_sequence, type= "Protein") 
                gene_associated = data[disease][species_type + '_gene'][i]
                gene_id = gene_associated[1]
                M.add_edge(protein_id, gene_id, type="Expression")

In [ ]:
color_map = {'Gene': 'blue', 'Protein': 'green', 'Disease': 'red'}

# Create a list of colors for the nodes based on their type
node_colors = [color_map[M.nodes[n]['type']] for n in M.nodes()]

# Draw the graph
nx.draw(M, with_labels=True, node_color=node_colors, font_color='white')

# Show the plot
plt.show()

In [38]:
import pickle as pkl

with open('data.pkl', 'wb') as f:
    pkl.dump(data, f)


In [39]:
print(data)

{'bipolar disorder': {'human_gene': [('BTBD16', 'ENSG00000138152'), ('ZNF804A', 'ENSG00000170396'), ('GPR50', 'ENSG00000102195'), ('NYX', 'ENSG00000188937'), ('ARK2C', 'ENSG00000141622'), ('ALG9', 'ENSG00000086848'), ('PCLO', 'ENSG00000186472'), ('SP4', 'ENSG00000105866')], 'mouse_gene': [('BTBD16', 'ENSMUSG00000040298'), ('ZNF804A', 'ENSMUSG00000070866'), ('GPR50', 'ENSMUSG00000056380'), ('NYX', 'ENSMUSG00000051228'), ('ARK2C', 'ENSMUSG00000025427'), ('ALG9', 'ENSMUSG00000032059'), ('PCLO', 'ENSMUSG00000061601'), ('SP4', 'ENSMUSG00000025323')], 'human_protein': [('BTBD16', 'Q32M84', 'MIMSNTHKARLERRVTGSTNRWRLPKQPFSGDLLSLSQMCKALSIDFEEALRNPDRLCISQIQKFFFENFKNKDIQSGEADVILECLGFKWELHQPQLFQSETLAKLYLKALAQGTTHPLRELEELLRAQSPKKTKEKSPAKRIIISLKINDPLVTKVAFATALKNLYMSEVEINLEDLLGVLASAHILQFSGLFQRCVDVMIARLKPSTIKKFYEAGCKYKEEQLTTGCEKWLEMNLVPLGGTQIHLHKIPQDLLHKVLKSPRLFTFSEFHLLKTMLLWVFLQLNYKIQAIPTYETVMTFFKSFPENCCFLDRDIGRSLRPLFLCLRLHGITKGKDLEVLRHLNFFPESWLDQVTVNHYHALENGGDMVHLKDLNTQAVRFGLLFNQENTTYSKTIALYGFFFKIKG